In [216]:
import numpy as np
import pandas as pd
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import geopy
from geopy import GoogleV3
import gmaps
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
import IPython
import json
import random
import sortedcontainers as smp
import csv
import time
#API key will be deleted upon submission, we can provide a new one on instructors request
API_KEY = 'AI...'


In [229]:
class Person:
    """description of class"""
    def __init__(self, first_name, last_name):
        #basic information about people
        self.first_name = first_name
        self.last_name = last_name
        self.middle_name = ""
        self.suffix = ""
        self.address = ""
        self.address_l1 = ""
        self.address_l2 = ""
        self.city = ""
        self.precinct = ""
        self.dob = ""
        self.zipcode = ""
        self.county = ""
        self.active = False

    def set_middle_name(self, middle_name):
        self.middle_name = middle_name

    def set_suffix(self, suffix):
        self.suffix = suffix

    def set_address_l1(self, address_l1):
        self.address_l1 = address_l1

    def set_address_l2(self, address_l2):
        self.address_l2 = address_l2

    def set_city(self, city):
        self.city = city

    def set_precinct(self, precinct):
        self.precinct = precinct

    def set_dob(self, dob):
        self.dob = dob

    def set_zipcode(self, zipcode):
        self.zipcode = zipcode

    def set_county(self, county):
        self.county = county

    def set_active(self, active):
        self.active = active

    def merge_addresses(self):
        self.address = self.address_l1.replace(' ', '') + self.address_l2.replace(' ', '')
        
class Party:
    #Fairly similar to the county class, but only keeps track of race and gender
    def __init__(self):
        self.races = { "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "9": 0}
        self.genders = { "M": 0, "F": 0, "U": 0}
        self.num_voters = 0
        self.race_percentages = { "1": 0.0, "2": 0.0, "3": 0.0, "4": 0.0, "5": 0.0, "6": 0.0, "7": 0.0, "9": 0.0}
        self.gender_percentages = { "M": 0.0, "F": 0.0, "U": 0.0}

    def process_data(self):
        #check in case some counties had no members of a certain party
        if self.num_voters > 0:
            for key in self.race_percentages:
                self.race_percentages[key] = float(self.races[key]) / float(self.num_voters) * 100
            for key in self.gender_percentages:
                self.gender_percentages[key] = float(self.genders[key]) / float(self.num_voters) * 100

    def display(self):
        print("Race Data: " + str(self.races))
        print("Number of Voters: " + str(self.num_voters))
        print("Percentage Race Data: ")
        print("American Indian or Alaskan Native: " + str("{:.2f}".format(self.race_percentages["1"])) + "%")
        print("Asian Or Pacific Islander: " + str("{:.2f}".format(self.race_percentages["2"])) + "%")
        print("Black, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["3"])) + "%")
        print("Hispanic: " + str("{:.2f}".format(self.race_percentages["4"])) + "%")
        print("White, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["5"])) + "%")
        print("Other: " + str("{:.2f}".format(self.race_percentages["6"])) + "%")
        print("Multi-racial: " + str("{:.2f}".format(self.race_percentages["7"])) + "%")
        print("Unknown: " + str("{:.2f}".format(self.race_percentages["9"])) + "%")
        print("Percentage Gender Data: ")
        print("Male: " + str("{:.2f}".format(self.gender_percentages["M"])) + "%")
        print("Female: " + str("{:.2f}".format(self.gender_percentages["F"])) + "%")
        print("Other: " + str("{:.2f}".format(self.gender_percentages["U"])) + "%")
        
class County:
    def __init__(self, file_name):
        #Most of the keys here are hard coded, taken straight from the document about which number represents which race, and what the party codes are
        self.countycode = file_name
        self.file_name = file_name + "_20201027.txt"
        self.races = { "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "7": 0, "9": 0}
        self.party_affiliations = { "CPF": Party(), "DEM": Party(), "ECO": Party(), "GRE": Party(), "IND": Party(), "LPF": Party(), "NPA": Party(), "PSL": Party(), "REF": Party(), "REP": Party()}
        self.genders = { "M": 0, "F": 0, "U": 0}
        self.total_voters = 0
        self.active_voters = 0
        self.race_percentages = { "1": 0.0, "2": 0.0, "3": 0.0, "4": 0.0, "5": 0.0, "6": 0.0, "7": 0.0, "9": 0.0}
        self.gender_percentages = { "M": 0.0, "F": 0.0, "U": 0.0}
        self.party_percentages = { "CPF": 0.0, "DEM": 0.0, "ECO": 0.0, "GRE": 0.0, "IND": 0.0, "LPF": 0.0, "NPA": 0.0, "PSL": 0.0, "REF": 0.0, "REP": 0.0}
        self.people_dict = {}
        self.people_sorted_dict = smp.SortedDict()

    #loads number of people associated with parties and races, also counts number of total voters and active voters
    def load_data(self):
        with open(self.file_name, "r") as county_file:
            reader = csv.reader(county_file, dialect="excel-tab")
            for row in reader:
                self.total_voters += 1
                if row[19] != "":
                    self.genders[row[19]] += 1
                self.races[row[20]] += 1               
                (self.party_affiliations[row[23]]).num_voters += 1
                (self.party_affiliations[row[23]]).races[row[20]] += 1
                if row[19] != "":
                    (self.party_affiliations[row[23]]).genders[row[19]] += 1
                if row[28] == "ACT":
                    self.active_voters += 1

    #stores the people in two dictionaries
    def store_people(self):
        with open(self.file_name, "r") as county_file:
            reader = csv.reader(county_file, dialect="excel-tab")
            for row in reader:
                #loads in everything lowercase to make search easier
                name = row[4] + " " + row[2]
                name = name.lower()
                temp_person = Person(row[4].lower(), row[2].lower())
                temp_person.set_middle_name(row[5].lower())
                temp_person.set_suffix(row[3].lower())
                temp_person.set_address_l1(row[7].lower())
                temp_person.set_address_l2(row[8].lower())
                temp_person.merge_addresses()
                temp_person.set_city(row[9].lower())
                temp_person.set_precinct(row[24].lower())
                temp_person.set_dob(row[21].lower())
                temp_person.set_zipcode(row[11])
                temp_person.set_county(self.countycode)
                if row[28] == "ACT":
                    temp_person.set_active(True)
                else:
                    temp_person.set_active(False)
                if self.people_dict.get(name, False) == False:
                    self.people_dict[name] = []
                self.people_dict[name].append(temp_person)
                self.people_sorted_dict.setdefault(name, [])
                self.people_sorted_dict[name].append(temp_person)
    

    #this is assuming name is "<First Name>" + " " + "<Last Name>"
    #Address is also assumed to be "<line 1>" + " " + "<line 2>"
    #returns precinct number
    #returns -1 if not found
     #this is assuming name is "<First Name>" + " " + "<Last Name>"
    #Address is also assumed to be "<line 1>" + " " + "<line 2>"
    #returns precinct number
    #returns -1 if not found
    def search_people_regular(self, name,  address,bday):
        name = name.lower()
        address = address.lower()
        #correction for 3 spaces in address from data
        address = address.replace(' ', '')
        address = address.replace(',', '')
        start_time = time.time()
        #check if voter exists
        if self.people_dict.get(name, False) == False:
             print("Error: Voter not found")
             time_taken = time.time() - start_time
             print("Search took " + str(time_taken) + " seconds for the Hashmap")
             return -1
        #voter's name is in the map    
        else:
            for person in self.people_dict[name]:
                if (person.address) == address:
                    #check to see if birthday is right
                    time_taken = time.time()- start_time
                    if(person.dob == bday):
                        print("Voter found.")
                        print("Search took " + str(time_taken) + " seconds for the Hashmap")
                        return person
        #voter name found but no matching address
        print("Error: Voter not found")
        time_taken = time.time() - start_time
        print("Search took " + str(time_taken) + " seconds for the Hashmap")
        return -1
    
    #same method but for the sorted dictionary
    def search_people_sorted(self, name, address,bday):
        name = name.lower()
        address = address.lower()
        #correction for 3 spaces in address from data
        address = address.replace(' ', '')
        address = address.replace(',', '')
        start_time = time.time()
        #check if voter exists
        if self.people_sorted_dict.get(name, False) == False:
             print("Error: Voter not found")
             time_taken = time.time() - start_time
             print("Search took " + str(time_taken) + " seconds for the tree based Map")
             return -1
        #voter's name is in the map    
        else:
            for person in self.people_sorted_dict[name]:
                if (person.address) == address:
                    #check to see if birthday is right
                    time_taken = time.time()- start_time
                    if(person.dob == bday):
                        print("Voter found.")
                        print("Search took " + str(time_taken) + " seconds for the tree based Map")
                        return person
        #voter name found but no matching address
        print("Error: Voter not found")
        time_taken = time.time() - start_time
        print("Search took " + str(time_taken) + " seconds for the tree based Map")
        return -1
        
    def process_data(self):
        #calculate race percentage in this county
        for key in self.race_percentages:
            self.race_percentages[key] = float(self.races[key]) / float(self.total_voters) * 100
        #calculate gender percentage in this county
        for key in self.gender_percentages:
            self.gender_percentages[key] = float(self.genders[key]) / float(self.total_voters) * 100
        #calculate party percentages in this county
        for key in self.party_percentages:
            self.party_percentages[key] = float(self.party_affiliations[key].num_voters) / float(self.total_voters) * 100
        #calculate the data for each party
        for key in self.party_affiliations:
            self.party_affiliations[key].process_data()

    #This method just takes the format from the .txt and hard codes the prints
    def display(self):
        print("Number of Voters: " + str(self.total_voters))
        print("Number of Active Voters: " + str(self.active_voters))
        print("Percentage Race Data: ")
        print("American Indian or Alaskan Native: " + str("{:.2f}".format(self.race_percentages["1"])) + "%")
        print("Asian Or Pacific Islander: " + str("{:.2f}".format(self.race_percentages["2"])) + "%")
        print("Black, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["3"])) + "%")
        print("Hispanic: " + str("{:.2f}".format(self.race_percentages["4"])) + "%")
        print("White, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["5"])) + "%")
        print("Other: " + str("{:.2f}".format(self.race_percentages["6"])) + "%")
        print("Multi-racial: " + str("{:.2f}".format(self.race_percentages["7"])) + "%")
        print("Unknown: " + str("{:.2f}".format(self.race_percentages["9"])) + "%")
        print("Percentage Party Data: ")
        print("Constitution Party of Florida: " + str("{:.2f}".format(self.party_percentages["CPF"])) + "%")
        print("Florida Democratic Party: " + str("{:.2f}".format(self.party_percentages["DEM"])) + "%")
        print("Ecology Party of Florida: " + str("{:.2f}".format(self.party_percentages["ECO"])) + "%")
        print("Green Party of Florida: " + str("{:.2f}".format(self.party_percentages["GRE"])) + "%")
        print("Independent Party of Florida: " + str("{:.2f}".format(self.party_percentages["IND"])) + "%")
        print("Libertarian Party of Florida: " + str("{:.2f}".format(self.party_percentages["LPF"])) + "%")
        print("No Party Affiliation: " + str("{:.2f}".format(self.party_percentages["NPA"])) + "%")
        print("Party for Socialism and Liberation - Florida: " + str("{:.2f}".format(self.party_percentages["PSL"])) + "%")
        print("Reform Party of Florida: " + str("{:.2f}".format(self.party_percentages["REF"])) + "%")
        print("Republican Party of Florida: " + str("{:.2f}".format(self.party_percentages["REP"])) + "%")
        print("Percentage Gender Data: ")
        print("Male: " + str("{:.2f}".format(self.gender_percentages["M"])) + "%")
        print("Female: " + str("{:.2f}".format(self.gender_percentages["F"])) + "%")
        print("Other: " + str("{:.2f}".format(self.gender_percentages["U"])) + "%")

    def create_file(self):
        f = open("statistics.txt", "a")
        f.write(str(self.countycode) + "\t")
        f.write("Number of Voters: " + str(self.total_voters) + "\t")
        f.write("Number of Active Voters: " + str(self.active_voters) + "\t")
        f.write("American Indian or Alaskan Native: " + str("{:.2f}".format(self.race_percentages["1"])) + "%" + "\t")
        f.write("Asian Or Pacific Islander: " + str("{:.2f}".format(self.race_percentages["2"])) + "%" + "\t")
        f.write("Black, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["3"])) + "%" + "\t")
        f.write("Hispanic: " + str("{:.2f}".format(self.race_percentages["4"])) + "%")
        f.write("White, Not Hispanic: " + str("{:.2f}".format(self.race_percentages["5"])) + "%" + "\t")
        f.write("Other: " + str("{:.2f}".format(self.race_percentages["6"])) + "%")
        f.write("Multi-racial: " + str("{:.2f}".format(self.race_percentages["7"])) + "%" + "\t")
        f.write("Unknown: " + str("{:.2f}".format(self.race_percentages["9"])) + "%" + "\t")
        f.write("Constitution Party of Florida: " + str("{:.2f}".format(self.party_percentages["CPF"])) + "%" + "\t")
        f.write("Florida Democratic Party: " + str("{:.2f}".format(self.party_percentages["DEM"])) + "%" + "\t")
        f.write("Ecology Party of Florida: " + str("{:.2f}".format(self.party_percentages["ECO"])) + "%" + "\t")
        f.write("Green Party of Florida: " + str("{:.2f}".format(self.party_percentages["GRE"])) + "%" + "\t")
        f.write("Independent Party of Florida: " + str("{:.2f}".format(self.party_percentages["IND"])) + "%" + "\t")
        f.write("Libertarian Party of Florida: " + str("{:.2f}".format(self.party_percentages["LPF"])) + "%" + "\t")
        f.write("No Party Affiliation: " + str("{:.2f}".format(self.party_percentages["NPA"])) + "%" + "\t")
        f.write("Party for Socialism and Liberation - Florida: " + str("{:.2f}".format(self.party_percentages["PSL"])) + "%" + "\t")
        f.write("Reform Party of Florida: " + str("{:.2f}".format(self.party_percentages["REF"])) + "%" + "\t")
        f.write("Republican Party of Florida: " + str("{:.2f}".format(self.party_percentages["REP"])) + "%" + "\t")
        f.write("Male: " + str("{:.2f}".format(self.gender_percentages["M"])) + "%" + "\t")
        f.write("Female: " + str("{:.2f}".format(self.gender_percentages["F"])) + "%" + "\t")
        f.write("Other: " + str("{:.2f}".format(self.gender_percentages["U"])) + "%" + "\t")
        f.write("\n")
        f.close()

In [230]:
def GeoLocation(address):
    """
    use for location.latitude, location.longitude, location.raw['formatted_address']
    """
    geo_locator = GoogleV3(api_key=API_KEY)
    location = geo_locator.geocode(address)
    return location

In [231]:
#has to be defined outside of class in order to use @capture functionality, which catches all output from a function being called (including functions that it calls)
out = widgets.Output(layout={'border': '1px solid black'})
out2 = widgets.Output(layout={'border': '1px solid black'})
class Sunshine:
    def __init__(self):
        #self.county_dict = {}
        self.counties = {'ALA': 'Alachua', 'BAK': 'Baker', 'BAY': 'Bay', 'BRA': 'Bradford', 'BRE': 'Brevard', 'BRO': 'Broward', 
                    'CAL': 'Calhoun', 'CHA': 'Charlotte', 'CIT': 'Citrus', 'CLA': 'Clay', 'CLL': 'Collier', 'CLM': 'Columbia', 
                    'DES': 'DeSoto', 'DIX': 'Dixie', 'DUV': 'Duval', 'ESC': 'Escambia', 'FLA': 'Flagler', 'FRA': 'Franklin', 
                    'GAD': 'Gadsden', 'GIL': 'Gilchrist', 'GLA': 'Glades', 'GUL': 'Gulf', 'HAM': 'Hamilton', 'HAR': 'Hardee', 
                    'HEN': 'Hendry', 'HER': 'Hernando', 'HIG': 'Highlands', 'HIL': 'Hillsborough', 'HOL': 'Holmes', 
                    'IND': 'Indian River', 'JAC': 'Jackson', 'JEF': 'Jefferson', 'LAF': 'Lafayette', 'LAK': 'Lake', 
                    'LEE': 'Lee', 'LEO': 'Leon', 'LEV': 'Levy', 'LIB': 'Liberty', 'MAD': 'Madison', 'MAN': 'Manatee', 
                    'MRN': 'Marion', 'MRT': 'Martin', 'DAD': 'Miami-Dade', 'MON': 'Monroe', 'NAS': 'Nassau', 
                    'OKA': 'Okaloosa', 'OKE': 'Okeechobee', 'ORA': 'Orange', 'OSC': 'Osceola', 'PAL': 'Palm Beach', 
                    'PAS': 'Pasco', 'PIN': 'Pinellas', 'POL': 'Polk', 'PUT': 'Putnam', 'STJ': 'St. Johns', 
                    'STL': 'St. Lucie', 'SAN': 'Santa Rosa', 'SAR': 'Sarasota', 'SEM': 'Seminole', 'SUM': 'Sumter', 
                    'SUW': 'Suwannee', 'TAY': 'Taylor', 'UNI': 'Union', 'VOL': 'Volusia', 'WAK': 'Wakulla', 
                    'WAL': 'Walton', 'WAS': 'Washington'}
       
        start = [27.9506, -82.4572]
        end = [29.6516, -82.3248]
        self.directions = gmaps.directions_layer(start, end, show_markers = False, show_route = False)
        
        self.title_widget = widgets.HTML(
          '<h3>Voting in the Sunshine</h3>'
          '<h4>Data from <a href="https://dos.myflorida.com/elections/data-statistics/voter-registration-statistics/voter-extract-disk-request/">Florida Department of Elections</a></h4>'
        )
        
        self.instructions_widget = widgets.HTML(
            '<p><i>Select your county and preferred search type below.</i></p>'
        )
        
        self.florida_map = self.init_colors()
        
        self.display_box = self.render_container()

        
    def init_search_select(self):
        search_select = widgets.Dropdown(
        options = [('Hash Search (Recommended)', 1), ('Tree Search', 2)],
        value =1,
        description='Search Type:',
        disabled = False,
    )
        return search_select
    
    def init_county_select(self):
        county_select = widgets.Dropdown(
        options = [('Alachua', 'ALA'), ('Baker', 'BAK'), ('Bay', 'BAY'), ('Bradford', 'BRA'), ('Brevard', 'BRE'), ('Broward', 'BRO'), ('Calhoun', 'CAL'), ('Charlotte', 'CHA'), ('Citrus', 'CIT'), ('Clay', 'CLA'), ('Collier', 'CLL'), ('Columbia', 'CLM'), ('DeSoto', 'DES'), ('Dixie', 'DIX'), ('Duval', 'DUV'), ('Escambia', 'ESC'), ('Flagler', 'FLA'), ('Franklin', 'FRA'), ('Gadsden', 'GAD'), ('Gilchrist', 'GIL'), ('Glades', 'GLA'), ('Gulf', 'GUL'), ('Hamilton', 'HAM'), ('Hardee', 'HAR'), ('Hendry', 'HEN'), ('Hernando', 'HER'), ('Highlands', 'HIG'), ('Hillsborough', 'HIL'), ('Holmes', 'HOL'), ('Indian River', 'IND'), ('Jackson', 'JAC'), ('Jefferson', 'JEF'), ('Lafayette', 'LAF'), ('Lake', 'LAK'), ('Lee', 'LEE'), ('Leon', 'LEO'), ('Levy', 'LEV'), ('Liberty', 'LIB'), ('Madison', 'MAD'), ('Manatee', 'MAN'), ('Marion', 'MRN'), ('Martin', 'MRT'), ('Miami-Dade', 'DAD'), ('Monroe', 'MON'), ('Nassau', 'NAS'), ('Okaloosa', 'OKA'), ('Okeechobee', 'OKE'), ('Orange', 'ORA'), ('Osceola', 'OSC'), ('Palm Beach', 'PAL'), ('Pasco', 'PAS'), ('Pinellas', 'PIN'), ('Polk', 'POL'), ('Putnam', 'PUT'), ('St. Johns', 'STJ'), ('St. Lucie', 'STL'), ('Santa Rosa', 'SAN'), ('Sarasota', 'SAR'), ('Seminole', 'SEM'), ('Sumter', 'SUM'), ('Suwannee', 'SUW'), ('Taylor', 'TAY'), ('Union', 'UNI'), ('Volusia', 'VOL'), ('Wakulla', 'WAK'), ('Walton', 'WAL'), ('Washington', 'WAS')],
        value = 'ALA',
        description = 'County:',
        disabled=False
    )
        return county_select
    
    def init_name_entry(self):
        name_box = widgets.Text(
        value = '',
        placeholder = 'Enter your name as First Last',
        description = 'Name:',
        disabled = False
    )
        return name_box
     
    def init_add_entry(self):
        add_box = widgets.Text(
        value = '',
        placeholder = 'Enter your street and apartment #. Road names must be abbreviated',
        description = 'Address:',
        layout = Layout(width='100%'))
        return add_box
    
    def init_dob_entry(self):
        dob_box = widgets.Text(
        value = '',
        placeholder = 'Enter in MM/DD/YYYY format',
        description = 'Date of Birth:')
        return dob_box
    
    def init_search_button(self):
        button = widgets.Button(
        description = "Search",
        disabled = False,
        button_style = 'success',
        tooltip = 'Search',
        icon = 'check'
    )
        button.on_click(self.search_button_click)
        return button
    
    def init_colors(self):
        gmaps.configure(api_key=API_KEY)
        fig = gmaps.figure(center = [27.6648,-81.5158], zoom_level = 6)
        with open("florida_counties.geojson") as f:
            counties = json.load(f)
        colors = []
        for feature in counties['features']:
            county_name = feature['properties']['county']
            try:
                #color = calculate_color(county_name)
                color = random.choice(['red', 'green', 'blue', 'purple', 'yellow', 'teal', (144, 55, 31)])
            except KeyError:
                color = (0, 0, 0, 0.3)
            colors.append(color)
        geojson = gmaps.geojson_layer(counties, fill_color = colors, stroke_color = colors, fill_opacity = 0.2)
        fig.add_layer(geojson)
        return fig
    
    #beginning of functionality
    @out.capture(clear_output = True)
    def search_button_click(self, b):
        if(self.search_control.value == 1):
            self.person = self.search_regular(self.name_field.value.lower().strip(), self.add_field.value.lower().strip(), self.county_control.value, 
                               self.dob_field.value.strip())
        else:
            self.person = self.search_sorted(self.name_field.value.lower().strip(), self.add_field.value.lower().strip(), self.county_control.value, 
                               self.dob_field.value.strip())
        try:
            self.assignedPrecinctDirections(self.person)
        except:
            print('Please revise your search terms')
            
         
    def search_regular(self, name, address, county, dob):
        working = County(county)
        working.store_people()
        person = working.search_people_regular(name, address, dob)
        self.display_county_stats(working)
        #free up memory between each search
        del working
        return person
    
    
    def search_sorted(self, name, address, county, dob):
        working = County(county)
        working.store_people()
        person = working.search_people_sorted(name, address, dob)
        self.display_county_stats(working)
        del working
        return person
    
    @out2.capture(clear_output = True)
    def display_county_stats(self, county):
        print('Voter Statstics for ' + self.counties[county.countycode] + ' county: ')
        county.load_data()
        county.process_data()
        county.display()
        
    def assignedPrecinctDirections(self, Person):
        #obtain person's location from their address
        person_location = GeoLocation(Person.address_l1+' '+Person.address_l2+ ' ' +Person.city+ ', FL ' + str(Person.zipcode))
        person_location_tuple = (person_location.latitude, person_location.longitude)
        # obtain person's precinct location from their precinct number
        precinct_address = ""
        file_name = "Precincts/"+ Person.county + "_precincts.txt"
        with open(file_name, "r") as precincts_file:
            reader = csv.reader(precincts_file, dialect="excel-tab")
            for row in reader:
                #loads in everything lowercase to make search easier
                if Person.precinct == row[0]:
                    precinct_address = row[2] + " " + row[3]
                    precinct_name = row[1]
                    
        precinct_location = GeoLocation(precinct_address)
        precinct_location_tuple = (precinct_location.latitude, precinct_location.longitude)
        
        self.directions.start = person_location_tuple
        self.directions.show_markers = True
        self.directions.show_route = True
        #Need to sleep to account for latency in accessing google's directions API, otherwise end will not be correctly updated
        time.sleep(.25)
        self.directions.end = precinct_location_tuple
        
        print("Your Precinct is: " + precinct_name)
        print('Precinct Address: ' + precinct_location.raw['formatted_address'])
        if(Person.active):
            print('Your voter status is ACTIVE')
        else:
            print('Your voter status is INACTIVE')
        
    
    def render_container(self):
        #making of county/search selections
        self.search_control = self.init_search_select()
        self.county_control = self.init_county_select()
        controls = widgets.HBox([self.search_control, self.county_control])
        
        #making of name/address inputs
        self.name_field = self.init_name_entry()
        self.add_field = self.init_add_entry()
        self.dob_field = self.init_dob_entry()
        entries = widgets.VBox([widgets.HBox([self.name_field, self.add_field]), self.dob_field])
        
        #search button
        button = self.init_search_button()
        
        
        #results and buttons
        search_results = widgets.HBox([widgets.VBox([button]), widgets.VBox([out, out2])])
        
        #directions
        self.florida_map.add_layer(self.directions)
        
        display_box = widgets.VBox(
          [self.title_widget, self.florida_map, self.instructions_widget, controls, entries, search_results],
          layout = Layout(height ='100%'))
        return display_box
    
    #rendering    
    def render(self):
        display(self.display_box)
        
    

In [232]:
#Display widget in this cell
program = Sunshine()
program.render()



In [227]:
#run this cell when done to free up memory
del program